In [6]:
import requests
from bs4 import BeautifulSoup
import os
import shutil
import time
import sys

### TODO:

- implement random genration of headers
- implement scrape all

In [7]:
class HTTP_Headers(object):
    pass

In [14]:
class OnePieceMangaScraper(object):
    
    def __init__(self):
        self.headers = headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        self.src_urls = {}
        self.base_url = 'http://onepiece-tube.com/kapitel/'
        
    def scrape_chapter(self, chapter_no):
        """"""
        image_urls = {}
        try:
            i = 1
            while True:
                response = requests.get(self.base_url + str(chapter_no) + '/' + str(i))
                soup = BeautifulSoup(response.text, 'html.parser')
                response = soup.find('div', attrs={'id':'resize'}).img['src']
                image_urls[i] = response
                i += 1
        except:
            sys.stdout.write(f'\rScraping stopped. End of Chapter reached at {i} pages.')
        
        self.src_urls[chapter_no] = image_urls
        self._extract_images(chapter_no=chapter_no)
        
        return self
          
    def scrape_all_chapters(self):
        """"""
        latest_chapter = self.get_latest_chapter()
        i = 1
        while i < latest_chapter:
            sys.stdout.flush()
            sys.stdout.write(f'\r ---- Currently Scraping Chapter {i} of {latest_chapter} ----')
            self.scrape_chapter(chapter_no=i)
            time.sleep(1)
            i += 1               
    
    def get_latest_chapter(self):
        """Gets the number of the latest chapter"""
        response = requests.get('http://onepiece-tube.com/kapitel-mangaliste#oben')
        soup = BeautifulSoup(response.text, 'html.parser')
        tabel_div = soup.find_all('div', attrs={'class':'sagatable'}) 
        all_rows = rows[0].find_all('tr')
        latest_chapter = rows[1].find('td').text
        
        return int(latest_chapter)                    
                             
    def _extract_images(self, chapter_no):
        """"""
        for page, url in self.src_urls[chapter_no].items():
            response = requests.get(url=url, stream=True, headers=self.headers)
            OnePieceMangaScraper._save_image(image_ressource=response, chapter_no=chapter_no, 
                                             page_number=page)

    
    @staticmethod
    def _save_image(image_ressource, chapter_no, page_number):
        """"""
        try:
            if 'OnePiece' not in os.listdir():
                os.mkdir('OnePiece') 
            os.mkdir('OnePiece/' + 'Chapter_' + str(chapter_no))
        except:
            pass
                 
        if image_ressource.status_code == 200:
            with open('OnePiece/Chapter_' + str(chapter_no) + '/Page_' + str(page_number) +'.png', 'wb') as f:
                image_ressource.raw.decode_content = True
                shutil.copyfileobj(image_ressource.raw, f)
                f.close()
        

In [17]:
OnePieceMangaScraper().scrape_chapter(300)

Scraping stopped. End of Chapter reached at 1 pages.

In [48]:
scraper = OnePieceMangaScraper()
scraper.scrape_all_chapters()

 ---- Currently Scraping Chapter 1 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 2 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 3 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 4 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 5 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 6 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 7 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 8 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 9 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 10 of 1018Scraping stopped. End of Chapter reached at 1 pages.
 ---- Currently Scraping Chapter 11 of 1018Scrapi

KeyboardInterrupt: 

In [15]:
response = requests.get('http://onepiece-tube.com/kapitel-mangaliste#oben')
soup = BeautifulSoup(response.text, 'html.parser')

In [22]:
tabel_div = soup.find_all('div', attrs={'class':'sagatable'})

In [26]:
all_rows = rows[0].find_all('tr')

In [41]:
print(first_row)

1018
